In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('data.csv')

# Get data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211833 entries, 0 to 211832
Data columns (total 25 columns):
 #   Column                                                              Non-Null Count   Dtype  
---  ------                                                              --------------   -----  
 0   id                                                                  211833 non-null  int64  
 1   Age (y)                                                             211833 non-null  int64  
 2   Gender(1, male; 2, female)                                          211833 non-null  int64  
 3   site                                                                211833 non-null  int64  
 4   height(cm)                                                          211831 non-null  float64
 5   weight(kg)                                                          211833 non-null  float64
 6   BMI(kg/m2)                                                          211833 non-null  float64
 7   SB

In [2]:
data.columns

Index(['id', 'Age (y)', 'Gender(1, male; 2, female)', 'site', 'height(cm)',
       'weight(kg)', 'BMI(kg/m2)', 'SBP(mmHg)', 'DBP(mmHg)', 'FPG (mmol/L)',
       'Cholesterol(mmol/L)', 'Triglyceride(mmol/L)', 'HDL-c(mmol/L)',
       'LDL(mmol/L)', 'ALT(U/L)', 'AST(U/L)', 'BUN(mmol/L)', 'CCR(umol/L)',
       'FPG of final visit(mmol/L)',
       'Diabetes diagnosed during followup（1,Yes）',
       'censor of diabetes at followup(1, Yes; 0, No)', 'year of followup',
       'smoking status(1,current smoker;2, ever smoker;3,never smoker)',
       'drinking status(1,current drinker;2, ever drinker;3,never drinker)',
       'family histroy of diabetes(1,Yes;0,No)'],
      dtype='object')

In [17]:
clean_data = data.copy()

# Remove all unnecessary columns
clean_data = clean_data.drop(columns=['id', 'site'])

# Rename columns to be more accessible
rename_dict = {
    'Age (y)': 'age',
    'Gender(1, male; 2, female)': 'sex',
    'height(cm)': 'height',
    'weight(kg)': 'weight',
    'BMI(kg/m2)': 'bmi',
    'SBP(mmHg)': 'sbp',
    'DBP(mmHg)': 'dbp',
    'FPG (mmol/L)': 'fpg',
    'Cholesterol(mmol/L)': 'chol',
    'Triglyceride(mmol/L)': 'tg',
    'HDL-c(mmol/L)': 'hdlc',
    'LDL(mmol/L)': 'ldl',
    'ALT(U/L)': 'alt',
    'AST(U/L)': 'ast',
    'BUN(mmol/L)': 'bun',
    'CCR(umol/L)': 'ccr',
    'FPG of final visit(mmol/L)': 'fpg_final',
    'Diabetes diagnosed during followup（1,Yes）': 'diabetes',
    'censor of diabetes at followup(1, Yes; 0, No)': 'censor',
    'year of followup': 'followup',
    'smoking status(1,current smoker;2, ever smoker;3,never smoker)': 'smoker',
    'drinking status(1,current drinker;2, ever drinker;3,never drinker)': 'drinker',
    'family histroy of diabetes(1,Yes;0,No)': 'fam_hist'
}
clean_data = clean_data.rename(columns=rename_dict)
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211833 entries, 0 to 211832
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   age        211833 non-null  int64  
 1   sex        211833 non-null  int64  
 2   height     211831 non-null  float64
 3   weight     211833 non-null  float64
 4   bmi        211833 non-null  float64
 5   sbp        211810 non-null  float64
 6   dbp        211809 non-null  float64
 7   fpg        211833 non-null  float64
 8   chol       206979 non-null  float64
 9   tg         206946 non-null  float64
 10  hdlc       117271 non-null  float64
 11  ldl        118412 non-null  float64
 12  alt        210051 non-null  float64
 13  ast        88543 non-null   float64
 14  bun        190282 non-null  float64
 15  ccr        200658 non-null  float64
 16  fpg_final  211814 non-null  float64
 17  diabetes   1304 non-null    float64
 18  censor     211833 non-null  int64  
 19  followup   211833 non-n

In [24]:
# Clean sex column
clean_data['sex'] = clean_data['sex'].replace({1: 'male', 2: 'female'})

# Check to see columns are all filled
print(f'Sexes in dataset: {clean_data["sex"].unique()}')

Sexes in dataset: ['female' 'male']


In [49]:
# Clean height column
print(f'Range of heights: {clean_data["height"].min()} - {clean_data["height"].max()}')
print(f'Number of nulls: {(clean_data.loc[clean_data["height"].isna()]).shape[0]}')

Range of heights: 106.5 - 198.5
Number of nulls: 2
